In [1]:
# Create input data to generate size term estimation for trip/tour destination choice

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load parcel data and MAZ lookups
df_parcel = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\v3_RTP\parcels_urbansim.txt', sep=' ')

In [3]:
parcel_block = pd.read_csv(r"R:\e2projects_two\activitysim\conversion\geographic_crosswalks\parcel_taz_block_lookup.csv")

In [4]:
df_parcel = df_parcel.merge(
            parcel_block, left_on="parcelid", right_on="parcel_id", how="left"
        )
df_parcel.rename(columns={"maz_id": "MAZ"}, inplace=True)

In [5]:
df_maz = df_parcel.groupby('MAZ').sum()
df_maz = df_maz.reset_index()
df_maz.rename(columns={'MAZ': 'zone'},inplace=True)

In [6]:
# Create a field that takes log of park area (aparks)
df_maz['log_aparks'] = np.log(1+df_maz['aparks'])

In [13]:
# Load Survey data
trip_df = pd.read_csv(r'\\modelstation2\c$\Workspace\psrc_activitysim_master\data\survey_data\override_trips.csv')
tour_df = pd.read_csv(r'\\modelstation2\c$\Workspace\psrc_activitysim_master\data\survey_data\override_tours.csv')
hh_df = pd.read_csv(r'\\modelstation2\c$\Workspace\psrc_activitysim_master\data\survey_data\override_households.csv')
person_df = pd.read_csv(r'\\modelstation2\c$\Workspace\psrc_activitysim_master\data\survey_data\override_persons.csv')

# Recode household income into categories
# Using quartiles from synthetic population
bins=[-np.inf, 43000, 82000, 135000, np.inf]
hh_df['income_bin'] = pd.cut(hh_df['income'], bins, labels=["work_low", "work_med","work_high", "work_very_high"])
hh_df['income_bin'] = hh_df['income_bin'].astype('str')

# Create some bins for student type
bins = [-np.inf, 14, 19, np.inf]
person_df['school_bin'] = pd.cut(person_df['age'], bins, labels=["gradeschool", "highschool","university"])

person_df = person_df.merge(hh_df, on='household_id', how='left')

In [14]:
# Calculate work location using usual location from person records; attach MAZ land use data

df_work = person_df[person_df['workplace_zone_id'] > 0]
df_work = df_work.pivot_table(index='workplace_zone_id', columns='income_bin', aggfunc='count', values='person_id')
df_work = df_work.fillna(0)
df_work = df_work.reset_index()

df_work = df_maz.merge(df_work, left_on='zone', right_on='workplace_zone_id', how='left')
df_work = df_work.fillna(0)

df_work.to_csv(r'size_term_17_19_workplace.csv')
df_work

,zone,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,...,xcoord_p,ycoord_p,parcel_id,taz_id,log_aparks,workplace_zone_id,work_high,work_low,work_med,work_very_high
0,1,87910,26,29,0,48,93,826,122,47,...,2.411154e+07,4.453337e+06,6658336,7676,11.38408,1.0,4.0,0.0,3.0,4.0
1,2,0,1,0,0,2,7,32,1,1,...,4.800623e+07,8.970497e+06,7261933,14706,0.00000,0.0,0.0,0.0,0.0,0.0
2,3,0,0,0,0,0,0,0,0,0,...,3.770623e+07,5.845713e+06,5989778,23700,0.00000,0.0,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,2.887193e+07,4.456404e+06,4629969,19205,0.00000,0.0,0.0,0.0,0.0,0.0
4,5,0,0,1,0,3,4,7,4,1,...,1.255819e+06,1.937765e+05,201116,834,0.00000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51199,66227,0,0,0,0,0,0,0,0,0,...,2.900987e+06,8.327804e+05,2586323,4218,0.00000,0.0,0.0,0.0,0.0,0.0
51200,66231,0,0,0,0,0,0,0,0,0,...,7.257960e+06,2.288579e+06,5461095,10545,0.00000,0.0,0.0,0.0,0.0,0.0
51201,66232,0,0,0,0,1,0,0,0,0,...,2.469887e+07,7.770543e+06,17919734,35853,0.00000,0.0,0.0,0.0,0.0,0.0
51202,66233,0,0,0,0,0,0,0,0,0,...,1.016433e+07,3.200145e+06,7556605,14763,0.00000,0.0,0.0,0.0,0.0,0.0


In [10]:
# Calculate school using usual location from person records; attach MAZ land use data

df_school = person_df[person_df['school_zone_id'] > 0]
df_school = df_school.pivot_table(index='school_zone_id', columns='school_bin', aggfunc='count', values='person_id')
df_school = df_school.fillna(0)
df_school = df_school.reset_index()

df_school = df_maz.merge(df_school, left_on='zone', right_on='school_zone_id', how='left')
df_school = df_school.fillna(0)

df_school.to_csv(r'size_term_17_19_school.csv')
df_school

,zone,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,...,taz_p,xcoord_p,ycoord_p,parcel_id,taz_id,log_aparks,school_zone_id,gradeschool,highschool,university
0,1,87910,26,29,0,48,93,826,122,47,...,7676,2.411154e+07,4.453337e+06,6658336,7676,11.38408,0.0,0.0,0.0,0.0
1,2,0,1,0,0,2,7,32,1,1,...,14706,4.800623e+07,8.970497e+06,7261933,14706,0.00000,0.0,0.0,0.0,0.0
2,3,0,0,0,0,0,0,0,0,0,...,23700,3.770623e+07,5.845713e+06,5989778,23700,0.00000,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,19205,2.887193e+07,4.456404e+06,4629969,19205,0.00000,0.0,0.0,0.0,0.0
4,5,0,0,1,0,3,4,7,4,1,...,834,1.255819e+06,1.937765e+05,201116,834,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51199,66227,0,0,0,0,0,0,0,0,0,...,4218,2.900987e+06,8.327804e+05,2586323,4218,0.00000,0.0,0.0,0.0,0.0
51200,66231,0,0,0,0,0,0,0,0,0,...,10545,7.257960e+06,2.288579e+06,5461095,10545,0.00000,0.0,0.0,0.0,0.0
51201,66232,0,0,0,0,1,0,0,0,0,...,35853,2.469887e+07,7.770543e+06,17919734,35853,0.00000,0.0,0.0,0.0,0.0
51202,66233,0,0,0,0,0,0,0,0,0,...,14763,1.016433e+07,3.200145e+06,7556605,14763,0.00000,0.0,0.0,0.0,0.0


In [9]:
# Use tour file to get non mandatory tour destinations
non_mand_df = tour_df[tour_df['tour_category'] == 'non_mandatory']
non_mand_df = non_mand_df.pivot_table(index='destination', columns='tour_type', values='tour_id', aggfunc='count')
non_mand_df = non_mand_df.reset_index()
non_mand_df = non_mand_df.fillna(0)
non_mand_df.rename(columns={'destination': 'maz'}, inplace=True)

# non_mand_df = df_maz.merge(non_mand_df, left_on='zone', right_on='maz', how='left')
# non_mand_df = non_mand_df.fillna(0)
#
# non_mand_df.to_csv('size_term_17_19_non_mandatory.csv')

non_mand_df

tour_type,maz,eatout,escort,othdiscr,othmaint,shopping,social
0,1,2.0,0.0,1.0,0.0,0.0,1.0
1,39,0.0,0.0,2.0,0.0,0.0,0.0
2,57,1.0,0.0,0.0,0.0,0.0,0.0
3,58,0.0,0.0,1.0,0.0,0.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3253,66011,0.0,0.0,0.0,0.0,0.0,1.0
3254,66019,0.0,2.0,0.0,0.0,0.0,0.0
3255,66031,0.0,0.0,0.0,0.0,1.0,0.0
3256,66099,0.0,0.0,1.0,1.0,3.0,0.0


In [18]:
# Calculate destinations for intermediate stops at the trip level

# Remove tour destinations from trip level to only use intermediate stops
# trip_df = trip_df.merge(tour_df, how='left', on='tour_id', suffixes=['_trip', '_tour'])
# trip_df2 = trip_df[trip_df['destination_trip'] != trip_df['destination_tour']].copy()
#
df = trip_df.pivot_table(index='destination_trip', columns='purpose', aggfunc='count', values='person_id_trip')
df = df.fillna(0)
df.drop(['Home','school'], axis=1, inplace=True)
df.rename(columns={'destination_trip': 'maz'}, inplace=True)

df = df_maz.merge(df, left_on='zone', right_on='destination_trip', how='left')
df = df.fillna(0)

df.to_csv('size_term_17_19_all_trip.csv')

df

,zone,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,...,taz_id,log_aparks,eatout,escort,othdiscr,othmaint,shopping,social,univ,work
0,1,87910,26,29,0,48,93,826,122,47,...,7676,11.38408,2.0,0.0,2.0,2.0,0.0,2.0,0.0,10.0
1,2,0,1,0,0,2,7,32,1,1,...,14706,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0,0,0,0,0,0,0,0,...,23700,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,19205,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,1,0,3,4,7,4,1,...,834,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51199,66227,0,0,0,0,0,0,0,0,0,...,4218,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51200,66231,0,0,0,0,0,0,0,0,0,...,10545,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51201,66232,0,0,0,0,1,0,0,0,0,...,35853,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51202,66233,0,0,0,0,0,0,0,0,0,...,14763,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(trip_df)

36668

In [12]:
# Classify atwork tours separately from school/work and non-mandatory

atwork_df = tour_df[tour_df['tour_category'] == 'atwork']
atwork_df =atwork_df.groupby('destination').count()[['person_id']].reset_index()
atwork_df.rename(columns={'person_id': 'atwork'},inplace=True)

atwork_df = df_maz.merge(atwork_df, left_on='zone', right_on='destination', how='left')
atwork_df = atwork_df.fillna(0)

atwork_df.to_csv('size_term_17_19_atwork.csv')

atwork_df

,zone,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,...,stuhgh_p,stuuni_p,taz_p,xcoord_p,ycoord_p,parcel_id,taz_id,log_aparks,destination,atwork
0,1,87910,26,29,0,48,93,826,122,47,...,0,0,7676,2.411154e+07,4.453337e+06,6658336,7676,11.38408,0.0,0.0
1,2,0,1,0,0,2,7,32,1,1,...,0,0,14706,4.800623e+07,8.970497e+06,7261933,14706,0.00000,0.0,0.0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,23700,3.770623e+07,5.845713e+06,5989778,23700,0.00000,0.0,0.0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,19205,2.887193e+07,4.456404e+06,4629969,19205,0.00000,0.0,0.0
4,5,0,0,1,0,3,4,7,4,1,...,0,0,834,1.255819e+06,1.937765e+05,201116,834,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51199,66227,0,0,0,0,0,0,0,0,0,...,0,0,4218,2.900987e+06,8.327804e+05,2586323,4218,0.00000,0.0,0.0
51200,66231,0,0,0,0,0,0,0,0,0,...,0,0,10545,7.257960e+06,2.288579e+06,5461095,10545,0.00000,0.0,0.0
51201,66232,0,0,0,0,1,0,0,0,0,...,0,0,35853,2.469887e+07,7.770543e+06,17919734,35853,0.00000,0.0,0.0
51202,66233,0,0,0,0,0,0,0,0,0,...,0,0,14763,1.016433e+07,3.200145e+06,7556605,14763,0.00000,0.0,0.0


purpose,eatout,escort,othdiscr,othmaint,shopping,social,univ,work
destination_trip,,,,,,,,
1,0.0,0.0,1.0,2.0,0.0,1.0,0.0,4.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
66099,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
66117,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
66123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
trip_df.head()

,trip_id,survey_trip_id,person_id_trip,household_id_trip,survey_tour_id_trip,outbound,purpose,destination_trip,origin_trip,depart,...,origin_tour,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency,stop_frequency
0,1588329.0,90.0,4842,3005,10.0,True,othmaint,3508,32658,9.0,...,32658,9.0,11.0,SHARED2FREE,NaN,NaN,adults,72,NaN,1out_0in
2,1588333.0,130.0,4842,3005,10.0,False,Home,32658,1057,10.0,...,32658,9.0,11.0,SHARED2FREE,NaN,NaN,adults,72,NaN,1out_0in
4,1588469.0,210.0,4842,3005,20.0,False,Home,32658,15792,15.0,...,32658,11.0,15.0,DRIVEALONEFREE,NaN,NaN,NaN,103,NaN,0out_0in
6,1589149.0,370.0,4844,4818,40.0,False,Home,25299,7163,11.0,...,25299,8.0,11.0,WALK,NaN,NaN,NaN,57,no_subtours,0out_0in
8,1589101.0,450.0,4844,4818,50.0,False,Home,25299,5173,13.0,...,25299,12.0,13.0,DRIVEALONEFREE,NaN,NaN,NaN,113,NaN,0out_0in
